# Tree with state propagation

In [ ]:
%matplotlib widget
import bmcs_utils.api as bu
import traits.api as tr
import numpy as np

## Eager versus lazy

The eager version uses the event trait variable and propagates the change to the parents. As a consequence, any assignment to an attribute tagged as state variable results in the state propagation through the dependency tree. For applications that change the state in an interation loop, like equilibrium iteration or optimization process, this leads to an excessive number of repeated calls. 

## How to avoid repeated state change calls

An event trait has no value. It can just be assigned. Therefore, it is impossible to detect if a change has already been handled by the parent or not. The goal is to allow for many changes in a child and let the parent react once there is a request from outside.

An example of this type is a material law with the material parameters as its attributes.

In [ ]:
scd = False

# Define a submodel

In [ ]:
class Rectangle(bu.Model):
    name = 'rectangle'
    
    length = bu.Float(2, GEO=True)
    width = bu.Float(2, GEO=True)

    A = tr.Property
    def _get_A(self):
        return self.length * self.width
    
    ipw_view = bu.View(
        bu.Item('length'),
        bu.Item('width')
    )
    
class Circle(bu.Model):
    name = 'circle'
    
    radius = bu.Float(2, GEO=True)

    A = tr.Property
    def _get_A(self):
        return self.radius**2 * np.pi

    ipw_view = bu.View(
        bu.Item('radius')
    )

In [ ]:
class ReinfLayer(bu.Model):
    name = 'layer'
    css = bu.EitherType(options=[('rectangle', Rectangle),
                                 ('circle', Circle)])
    
    A = tr.Property(bu.Float, depends_on='state_changed')
    @tr.cached_property
    def _get_A(self):
        return self.css_.A

    depends_on = ['css_']
    ipw_view = bu.View(
        bu.Item('css'),
        bu.Item('A')
    )

In [ ]:
rl = ReinfLayer()
rl.state_change_debug = scd
# rl.interact()

In [ ]:
changes = [('rectangle', 'length', 3),
           ('circle', 'radius', 3),
           ('rectangle', 'length', 5),
           ('circle', 'radius', 3),
           ('rectangle', 'length', 5),
           ]
for css_type, attr, value in changes:
    print('-----------------------')
    rl.reset_state_change()
    print(css_type, attr, value)
    rl.css = css_type
    setattr(rl.css_, attr, value)
    print('A', rl.A)
    print('state_changes', rl.state_change_counter)

# Test a dictionary

In [ ]:
class CrossSectionLayout(bu.ModelDict):
    name = 'Cross Section Layout'
    

In [ ]:
csl = CrossSectionLayout()
csl.state_change_debug = False
csl['one'] = ReinfLayer(css='rectangle')
csl['two'] = ReinfLayer(css='circle')

In [ ]:
class Beam(bu.Model):
    name = 'Beam'
    csl = bu.Instance(CrossSectionLayout, ())
    tree = ['csl']
    
    sum_A = tr.Property(bu.Float, depends_on='state_changed')
    @tr.cached_property
    def _get_sum_A(self):
        return sum( rl.A for rl in self.csl.items.values() ) 

    ipw_view = bu.View(
        bu.Item('sum_A', readonly=True)
    )

In [ ]:
b = Beam(csl=csl)
b.interact()

In [ ]:
changes = [('rectangle', 'length', 3),
           ('circle', 'radius', 3),
           ('rectangle', 'length', 5),
           ('circle', 'radius', 5),
           ('rectangle', 'length', 5),
           ]
for css_type, attr, value in changes:
    rl.reset_state_change()
    print('-----------------------')
    rl = b.csl['one']
    print(css_type, attr, value)
    rl.css = css_type
    setattr(rl.css_, attr, value)
    print('A', rl.A)
    print('state_changes', rl.state_change_counter)

In [ ]:
b.interact()